In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd, datetime
import numpy as np
import requests
import time
from scipy.stats import linregress
from ydata_profiling import ProfileReport
from pathlib import Path


# Import the OpenWeatherMap API key
from api_keys import airpollution_api_key 


# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# airpollution_api_key = airpollution_api_key or "YOUR_API_KEY"
print(airpollution_api_key)

In [ ]:
# Load the CSV file into a Pandas DataFrame
city_health = pd.read_csv("../Resources/CDC_500_City_Health.csv")

# Display the data table for preview
city_health

In [ ]:
# review data types
city_health.info()

In [ ]:
# describe data
city_health.describe()

In [ ]:
# Run the Profiling Report using ydata_profiling
profile = ProfileReport(city_health,title = "City Health Profile Report")
profile.to_notebook_iframe()

In [ ]:
# drop columns with null values
clean_city_health = city_health.drop(['Data_Value_Footnote_Symbol','Data_Value_Footnote','TractFIPS'],axis=1)

In [ ]:
# review data types after dropping null values
clean_city_health.info()

In [ ]:
# describe data after dropping null values
clean_city_health.describe()

In [ ]:
# Run the Profiling Report using ydata_profiling after dropping null values
profile = ProfileReport(clean_city_health,title = "City Health Profile Report")
profile.to_notebook_iframe()

In [ ]:
# Review the number of cities per state per unique MeasureID
diseases_per_state = clean_city_health.groupby('StateDesc')['MeasureId'].value_counts()
diseases_per_state


In [ ]:
# total count of diseases per state
diseases_per_state_total = clean_city_health.groupby('StateDesc')['MeasureId'].count().sort_values(ascending=False)
diseases_per_state_total



In [ ]:
# number of cities per state with diseases
cities_per_state = clean_city_health.groupby('StateDesc')['CityName'].nunique().sort_values(ascending=False)
cities_per_state

In [ ]:
# change data type for Latitude and Longitude
clean_city_health[['Latitude', 'Longitude']] = clean_city_health['GeoLocation'].str.extract(r'\(([^,]+),([^)]+)\)', expand=True)
clean_city_health['Latitude'] = clean_city_health['Latitude'].astype(float)
clean_city_health['Longitude'] = clean_city_health['Longitude'].astype(float)
clean_city_health

In [ ]:
# sort values by CityName
clean_city_health = clean_city_health.sort_values(by='CityName')
clean_city_health

In [ ]:
# Review the data types
clean_city_health.info()

In [ ]:
# create a dataframe for cancer data eliminating the index
cancer_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'CANCER']
cancer_df.reset_index(drop=True, inplace=True)
cancer_df


In [ ]:
# Create a dataframe for copd data eliminating the index
copd_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'COPD']
copd_df.reset_index(drop=True, inplace=True)
copd_df

In [ ]:
# create a dataframe for asthma data eliminating the index
casthma_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'CASTHMA']
casthma_df.reset_index(drop=True, inplace=True)
casthma_df 

In [ ]:
# create a dataframe for chd data eliminating the index
chd_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'CHD']
chd_df.reset_index(drop=True, inplace=True)
chd_df


In [ ]:
city_coordinates = clean_city_health['GeoLocation'].drop_duplicates()
city_coordinates

In [ ]:
all_cities_air_pollution = []
for i in city_coordinates:
    clean_lat_lng = i.replace('(','').replace(')','').split(',')
    lon = clean_lat_lng[1].strip()
    # get data api for each lat and lng
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={clean_lat_lng[0]}&lon={lon}&appid={airpollution_api_key}'
    
    cityap_data = requests.get(url).json()
    coordinates = cityap_data['coord']
    components = cityap_data['list'][0]['components']
    all_cities_air_pollution.append([coordinates['lat'], coordinates['lon'], components['co'], components['no'], components['no2'], components['o3'], components['so2'], components['pm2_5'], components['pm10'], components['nh3']])


all_cities_air_pollution

In [ ]:
all_cities_air_pollution_df = pd.DataFrame(all_cities_air_pollution, columns=['Latitude', 'Longitude', 'CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10', 'NH3'])
all_cities_air_pollution_df

In [ ]:

all_cities_air_pollution_df.info()  

In [ ]:
# merge the cancer data with the air pollution data
cancer_air_pollution_df  = pd.concat([cancer_df, all_cities_air_pollution_df], axis=1)
cancer_air_pollution_df 

In [ ]:
# save the cancer_air_pollution_df to a csv file
cancer_air_pollution_df.to_csv('../air_pollution_cleaned_data/cancer_air_pollution.csv', index=False)

In [ ]:
# merge the copd data with the air pollution data
copd_air_pollution_df  = pd.concat([copd_df, all_cities_air_pollution_df], axis=1)
copd_air_pollution_df 

In [ ]:
# save the copd_air_pollution_df to a csv file
copd_air_pollution_df.to_csv('../air_pollution_cleaned_data/copd_air_pollution.csv', index=False)


In [ ]:
# merge the asthma data with the air pollution data
casthsma_air_pollution_df  = pd.concat([casthma_df, all_cities_air_pollution_df], axis=1)
casthsma_air_pollution_df 

In [ ]:
# save the casthsma_air_pollution_df to a csv file
casthsma_air_pollution_df.to_csv('../air_pollution_cleaned_data/casthsma_air_pollution.csv', index=False)


In [ ]:
# merge the chd data with the air pollution data
chd_air_pollution_df  = pd.concat([chd_df, all_cities_air_pollution_df], axis=1)
chd_air_pollution_df 

In [ ]:
# save the chd_air_pollution_df to a csv file
chd_air_pollution_df.to_csv('../air_pollution_cleaned_data/chd_air_pollution.csv', index=False)
